## Create Tables in Data Base

Tutorial [Link](https://colab.research.google.com/github/GoogleCloudPlatform/cloud-sql-python-connector/blob/main/samples/notebooks/mysql_python_connector.ipynb#scrollTo=wfEvH386zX2V)

In [2]:
# !gcloud sql databases create kexpDB --instance=kexp-sandbox

In [1]:
import google.auth

credentials, project_id = google.auth.default()

# install dependencies
# pip install cloud-sql-python-connector["pymysql"] SQLAlchemy==2.0.7

import sys
if '..' not in sys.path:
    sys.path.append('..')

from scripts.utils import openfile


from google.cloud.sql.connector import Connector
import sqlalchemy
from sqlalchemy import MetaData
from creds import user

In [2]:
# initialize parameters
INSTANCE_CONNECTION_NAME = user['INSTANCE_CONNECTION_NAME']# i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = user['DB_USER']
DB_PASS = user['DB_PASS']
DB_NAME = user['DB_NAME']


root_path = user['root_path']

Your instance connection name is: kexp-explore:us-central1:kexp-sandbox


In [3]:


# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

engine = pool

In [4]:
# Create a MetaData object
metadata = MetaData()

# Reflect the existing database schema into the MetaData object
metadata.reflect(bind=engine)

# Access information about tables in the 'tables' dictionary attribute
for table_name, table in metadata.tables.items():
    print(f"Table name: {table_name}")
    # Access columns, constraints, etc. through the 'table' object


Table name: hosts
Table name: plays
Table name: programs
Table name: shows


In [5]:
metadata.tables.items()

dict_items([('hosts', Table('hosts', MetaData(), Column('id', INTEGER(), table=<hosts>, primary_key=True, nullable=False), Column('uri', VARCHAR(length=255), table=<hosts>, nullable=False), Column('name', VARCHAR(length=100), table=<hosts>, nullable=False), Column('image_uri', VARCHAR(length=255), table=<hosts>), Column('thumbnail_uri', VARCHAR(length=255), table=<hosts>), Column('is_active', TINYINT(display_width=1), table=<hosts>, nullable=False), schema=None)), ('plays', Table('plays', MetaData(), Column('id', INTEGER(), table=<plays>, primary_key=True, nullable=False), Column('uri', VARCHAR(length=255), table=<plays>, nullable=False), Column('airdate', TIMESTAMP(), table=<plays>, nullable=False), Column('show_id', INTEGER(), table=<plays>, nullable=False), Column('song', VARCHAR(length=255), table=<plays>, nullable=False), Column('track_id', VARCHAR(length=255), table=<plays>, nullable=False), Column('recording_id', VARCHAR(length=255), table=<plays>, nullable=False), Column('artis

In [6]:


# connect to connection pool
with pool.connect() as db_conn:
  # create hosts table in our kexp-sandbox database
  db_conn.execute(sqlalchemy.text(openfile(root_path+'hosts.sql')))

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()

  # create plays table in our kexp-sandbox database
  db_conn.execute(sqlalchemy.text(openfile(root_path+'plays.sql')))

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()

  # create programs table in our kexp-sandbox database
  db_conn.execute(sqlalchemy.text(openfile(root_path+'programs.sql')))

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()

  # create shows table in our kexp-sandbox database
  db_conn.execute(sqlalchemy.text(openfile(root_path+'shows.sql')))

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()

  # create timeslots table in our kexp-sandbox database
  db_conn.execute(sqlalchemy.text(openfile(root_path+'timeslots.sql')))

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()


File contents:
CREATE TABLE IF NOT EXISTS hosts (
                id INT PRIMARY KEY,
                uri VARCHAR(255) UNIQUE NOT NULL,
                name VARCHAR(100) UNIQUE NOT NULL,
                image_uri VARCHAR(255),
                thumbnail_uri VARCHAR(255),
                is_active BOOL NOT NULL
            );
File contents:
CREATE TABLE IF NOT EXISTS plays (
                id INT PRIMARY KEY,
                uri VARCHAR(255) UNIQUE NOT NULL,
                airdate TIMESTAMP NOT NULL,
                show_id INT NOT NULL,
                song VARCHAR(255) NOT NULL,
                track_id VARCHAR(255) NOT NULL,
                recording_id VARCHAR(255) NOT NULL,
                artist VARCHAR(255) NOT NULL,
                album VARCHAR(255) NOT NULL,
                release_id VARCHAR(255),
                release_group_id VARCHAR(255),
                release_date VARCHAR(255),
                is_local BOOL,
                is_request BOOL,
                is_live BO

In [7]:
# Reflect the existing database schema into the MetaData object
metadata.reflect(bind=engine)

for t in metadata.tables.items():
    print(t[0])

hosts
plays
programs
shows
timeslots


## Populate Tables